### 환경세팅하기

In [414]:
import pandas as pd
import os

rating_file_path = os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head(10)

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [415]:
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size):{orginal_data_size},filtered_data_size:{filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size):1000209,filtered_data_size:836478
Ratio of Remaining Data is 83.63%


In [416]:
ratings.rename(columns={'rating':'play'}, inplace=True)
ratings

,user_id,movie_id,play,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [417]:
movie_file_path=os.getenv('HOME')+'/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id','title','genre']
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head(10)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


### 2) 분석해 봅시다.

#### ratings에 있는 유니크한 영화 개수
#### rating에 있는 유니크한 사용자 수
#### 가장 인기있는 영화 30개(인기순)

In [418]:
ratings.movie_id.count()

836478

In [419]:
#영화개수
ratings['movie_id'].nunique()

3628

In [420]:
#사용자 수
ratings['user_id'].nunique()

6039

In [421]:
#인기많은 영화 30개
movie_count = ratings.groupby('movie_id')['play'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: play, dtype: int64

In [422]:
#movie_id와 title 데이터 프레임 합치기
#모든 데이터 합치기. 
data_movie = pd.merge(left = ratings, right = movies, how = "outer", on = "movie_id")
data_movie

,user_id,movie_id,play,timestamp,title,genre
0,1.0,1193,5.0,978300760.0,One Flew Over the Cuckoo's Nest (1975),Drama
1,2.0,1193,5.0,978298413.0,One Flew Over the Cuckoo's Nest (1975),Drama
2,12.0,1193,4.0,978220179.0,One Flew Over the Cuckoo's Nest (1975),Drama
3,15.0,1193,4.0,978199279.0,One Flew Over the Cuckoo's Nest (1975),Drama
4,17.0,1193,5.0,978158471.0,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
836728,NaN,3829,NaN,NaN,Mad About Mambo (2000),Comedy|Romance
836729,NaN,3856,NaN,NaN,Autumn Heart (1999),Drama
836730,NaN,3891,NaN,NaN,Turn It Up (2000),Crime|Drama
836731,NaN,3904,NaN,NaN,"Uninvited Guest, An (2000)",Drama


In [423]:
#결측값 제거하기
data_movie = data_movie.dropna()

In [424]:
#movie name which is popular
movie_popular = data_movie.groupby('title')['play'].count()
movie_popular.sort_values(ascending=False).head(100)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
                                                         ... 
Graduate, The (1967)                                     1223
There's Something About Mary (1998)                      1223
Chicken Run (2000)                                       1223
Erin Brockovich (2000)                                   1223
Rock, The (1996)                                         1213
Name: play, Length: 100, dtype: int64

In [425]:
data_movie = data_movie.drop('timestamp', axis = 1)
data_movie = data_movie.drop('genre', axis = 1)

In [426]:
#소문자로 만들기
data_movie['title'] = data_movie['title'].str.lower()
data_movie.head(5)

,user_id,movie_id,play,title
0,1.0,1193,5.0,one flew over the cuckoo's nest (1975)
1,2.0,1193,5.0,one flew over the cuckoo's nest (1975)
2,12.0,1193,4.0,one flew over the cuckoo's nest (1975)
3,15.0,1193,4.0,one flew over the cuckoo's nest (1975)
4,17.0,1193,5.0,one flew over the cuckoo's nest (1975)


#### 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [427]:
my_favorite = ['21gram(2002)', 'innocence(2001)', 'mad max(2015)', 'jerry maquire(1998)', 'dark night(2005)']

In [428]:
my_list = pd.DataFrame({'user_id': [6041]*5, 'play': [5]*5, 'movie_id':[3953.0, 3954.0, 3955.0, 3956.0, 3957.0 ] , 'title': my_favorite})
my_list

,user_id,play,movie_id,title
0,6041,5,3953.0,21gram(2002)
1,6041,5,3954.0,innocence(2001)
2,6041,5,3955.0,mad max(2015)
3,6041,5,3956.0,jerry maquire(1998)
4,6041,5,3957.0,dark night(2005)


In [429]:
[6041]*5

[6041, 6041, 6041, 6041, 6041]

In [430]:
if not data_movie.isin({'user_id':[6041]})['user_id'].any():  # user_id에 'young'이라는 데이터가 없다면
    data_movie = data_movie.append(my_list)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data_movie.tail(10)    

,user_id,movie_id,play,title
836473,5851.0,3607.0,5.0,one little indian (1973)
836474,5854.0,3026.0,4.0,slaughterhouse (1987)
836475,5854.0,690.0,3.0,"promise, the (versprechen, das) (1994)"
836476,5938.0,2909.0,4.0,"five wives, three secretaries and me (1998)"
836477,5948.0,1360.0,5.0,identification of a woman (identificazione di ...
0,6041.0,3953.0,5.0,21gram(2002)
1,6041.0,3954.0,5.0,innocence(2001)
2,6041.0,3955.0,5.0,mad max(2015)
3,6041.0,3956.0,5.0,jerry maquire(1998)
4,6041.0,3957.0,5.0,dark night(2005)


In [431]:
data_movie['movie_id'].max()

3957.0

In [432]:
3739 in movie_idx_to_unique.values()

False

#### 4) CSR matrix를 직접 만들어 봅시다.

In [433]:
user_idx_to_unique = {v: k for k, v in enumerate(data_movie.user_id.unique())}
movie_idx_to_unique = {v: k for k, v in enumerate(data_movie.movie_id.unique())}
print(movie_idx_to_unique)

{1193.0: 0, 661.0: 1, 914.0: 2, 3408.0: 3, 2355.0: 4, 1197.0: 5, 1287.0: 6, 2804.0: 7, 594.0: 8, 919.0: 9, 595.0: 10, 938.0: 11, 2398.0: 12, 2918.0: 13, 1035.0: 14, 2791.0: 15, 2687.0: 16, 2018.0: 17, 3105.0: 18, 2797.0: 19, 2321.0: 20, 720.0: 21, 1270.0: 22, 527.0: 23, 2340.0: 24, 48.0: 25, 1097.0: 26, 1721.0: 27, 1545.0: 28, 745.0: 29, 2294.0: 30, 3186.0: 31, 1566.0: 32, 588.0: 33, 1907.0: 34, 783.0: 35, 1836.0: 36, 1022.0: 37, 2762.0: 38, 150.0: 39, 1.0: 40, 1961.0: 41, 1962.0: 42, 2692.0: 43, 260.0: 44, 1028.0: 45, 1029.0: 46, 1207.0: 47, 2028.0: 48, 531.0: 49, 3114.0: 50, 608.0: 51, 1246.0: 52, 1357.0: 53, 3068.0: 54, 1537.0: 55, 647.0: 56, 2194.0: 57, 648.0: 58, 2268.0: 59, 2628.0: 60, 1103.0: 61, 2916.0: 62, 3468.0: 63, 1210.0: 64, 1792.0: 65, 1687.0: 66, 3578.0: 67, 2881.0: 68, 3030.0: 69, 1217.0: 70, 2126.0: 71, 3108.0: 72, 3035.0: 73, 1253.0: 74, 1610.0: 75, 292.0: 76, 2236.0: 77, 3071.0: 78, 368.0: 79, 1259.0: 80, 3147.0: 81, 1544.0: 82, 1293.0: 83, 1188.0: 84, 3255.0: 85, 3

In [434]:
temp_user_data = data_movie.user_id.map(user_idx_to_unique.get).dropna()
temp_movie_data = data_movie.movie_id.map(movie_idx_to_unique.get).dropna()
print(temp_movie_data)

0       0
1       0
2       0
3       0
4       0
     ... 
0    3628
1    3629
2    3630
3    3631
4    3632
Name: movie_id, Length: 836483, dtype: int64


In [435]:
data_movie.user_id = temp_user_data
data_movie.movie = temp_movie_data

/home/goldhanwool/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [436]:
from scipy.sparse import csr_matrix

num_user = data_movie['user_id'].nunique()
num_movie = data_movie['movie_id'].nunique()


## 확인
csr_data = csr_matrix((data_movie.play, (data_movie.user_id, data_movie.movie_id.astype(int))))
csr_data

<6040x3958 sparse matrix of type '<class 'numpy.float64'>'
	with 836483 stored elements in Compressed Sparse Row format>

#### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [437]:
from implicit.als import AlternatingLeastSquares
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

#### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [438]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)
print('a')

a


In [439]:
# als 모델은 input으로(item X user 꼴의 matrix를 받기 때문에 Transpose 해준다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3958x6040 sparse matrix of type '<class 'numpy.float64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [440]:
# 모델 훈련
als_model.fit(csr_data_transpose)

In [441]:
young, innocence = user_idx_to_unique[6041.0], movie_idx_to_unique [3954]

In [442]:
print(young)

6039


In [443]:
young_vector, innocence_vector = als_model.user_factors[young], als_model.item_factors[innocence]

In [444]:
young_vector

array([-0.01259114,  0.00177916,  0.00376282,  0.01937305,  0.01100119,
        0.00921326,  0.00296128, -0.00845105, -0.02230633, -0.01624733,
        0.00849432,  0.00212746, -0.00230958, -0.00200916, -0.01410863,
        0.01637352,  0.00865134, -0.00890042,  0.01087905, -0.01407258,
       -0.00042029,  0.01332388,  0.00133411, -0.01941209,  0.00772638,
       -0.01208879, -0.00377444, -0.0067024 , -0.0044545 , -0.00339277,
        0.0130886 , -0.00237074,  0.01465186,  0.02920102,  0.00960751,
       -0.00322614, -0.00402683,  0.00288786, -0.010149  ,  0.00974153,
        0.01505856, -0.01055237,  0.00015098, -0.00485009, -0.02696163,
        0.00626   ,  0.00747949, -0.02140999,  0.00678554, -0.00566063,
       -0.01133129,  0.00991937, -0.00810852, -0.00341831, -0.00259172,
       -0.01344768, -0.00830866, -0.0084981 , -0.00815811,  0.0113934 ,
        0.00124598, -0.0207618 ,  0.00202988,  0.00302629, -0.00698066,
        0.0044424 , -0.00632385,  0.01549129,  0.00014955,  0.00

In [445]:
innocence_vector

array([ 0.01217118,  0.00780137,  0.00163231, -0.00283156,  0.02647502,
        0.00776701,  0.01776189,  0.00355793,  0.01490454, -0.00811312,
       -0.00959238, -0.00306163, -0.00549251,  0.00514555,  0.00088   ,
        0.01225087,  0.00899868,  0.01823185,  0.01992546,  0.00801564,
        0.00395353, -0.00415743,  0.00579333,  0.01849301,  0.01735522,
        0.00360011,  0.03368995,  0.01921011,  0.01486898, -0.01066597,
       -0.02113584,  0.01550714,  0.00389142,  0.01516959,  0.01596986,
       -0.01643265, -0.00160559, -0.00815819,  0.00778424,  0.0072218 ,
        0.00073786,  0.01380786, -0.01761136,  0.00378765,  0.01187292,
        0.02644409, -0.00672761, -0.00081046,  0.00337621,  0.01885777,
       -0.0073767 ,  0.01024787,  0.00721837,  0.02470968, -0.01340194,
        0.01514371,  0.00144081,  0.02170672,  0.00807022,  0.00633446,
        0.00264972, -0.01096052,  0.0033404 ,  0.0221419 ,  0.01756351,
       -0.00543312, -0.01101977,  0.03187341, -0.00873375,  0.01

In [446]:
np.dot(young_vector, innocence_vector)

0.00062472996

In [447]:
young, mad_max= user_idx_to_unique[6041.0],  movie_idx_to_unique[3955]
young_vector, mad_max_vector = als_model.user_factors[young], als_model.item_factors[mad_max]

In [448]:
np.dot(young_vector, toy_story_vector)

1.5031001e-06

#### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [449]:
favorite_movie = movie_idx_to_unique[3957]
similar_movie = als_model.similar_items(favorite_movie, N=15)
similar_movie

[(3632, 1.0000001),
 (970, 0.80874765),
 (3739, 0.79335195),
 (3306, 0.7830693),
 (1067, 0.7576544),
 (3640, 0.75658536),
 (946, 0.75440115),
 (2847, 0.7478806),
 (3532, 0.74210703),
 (2936, 0.73228896),
 (3133, 0.73207635),
 (2927, 0.72965986),
 (1934, 0.7295573),
 (958, 0.7260881),
 (1068, 0.72427905)]

In [450]:
idx_to_movie = {v:k for k,v in movie_idx_to_unique.items()}
[idx_to_movie[i[0]] for i in similar_movie]

KeyError: 3739

#### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [451]:
young = user_idx_to_unique[6041.0]
movie_recommend = als_model.recommend(young, csr_data, N=20, filter_already_liked_items=True)
print('a')

a


In [452]:
idx_to_movie = {v:k for k,v in movie_idx_to_unique.items()}
[idx_to_movie[i[0]] for i in movie_recommend]

KeyError: 3948

In [453]:
explain = als_model.explain(young, csr_data, itemid = innocence)
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

KeyError: 3955